In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

url = 'https://baseball.yahoo.co.jp/npb/game/2021005991/score?index=0110100'

#データフレームを用意
result = pd.DataFrame()

#1秒待機
time.sleep(1)
#read_htmlでデータを取得
df = pd.read_html(url, encoding='utf-8')
#投球数、球種、球速、結果、投手、打者、スコアを取得
result = df[14].drop('投球数.1', axis=1)
result['投手'] = df[13].loc[0, '投手']
result['打者'] = df[13].loc[0, '打者']
result['スコア'] = '-'.join(map(str, df[1].iloc[:, 1].tolist()))

#1秒待機
time.sleep(1)
#BeautifulSoupでデータを取得
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
#投球コースを取得
Chart = soup.find('div', class_= 'bb-allocationChart').find_all('span', class_='bb-icon__ballCircle')
chart = [c['style'] for c in Chart]
result['投球コース'] = chart

In [2]:
result
#topは上から何pxかどうか
#leftは画面左から何pxかどうか

,投球数,球種,球速,結果,投手,打者,スコア,投球コース
0,1,ストレート,146km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:11px; left:132px;
1,2,ストレート,146km/h,見逃し,渡邉 勇太朗,辰己 涼介,0-0,top:119px; left:112px;
2,3,スライダー,130km/h,ファウル,渡邉 勇太朗,辰己 涼介,0-0,top:35px; left:72px;
3,4,ストレート,149km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:150px; left:99px;
4,5,カットボール,135km/h,左失,渡邉 勇太朗,辰己 涼介,0-0,top:88px; left:68px;


In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

url = 'https://baseball.yahoo.co.jp/npb/game/2021005991/score?index=0110100'
#データフレームを用意
df_result = pd.DataFrame()

for i in range(1000):
    #1秒待機
    time.sleep(1)
    #read_htmlでデータを取得
    df = pd.read_html(url, encoding='utf-8')
    #BeautifulSoupでデータを取得
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    #「次へ」ボタンからURLを取得
    try:
        next = soup.find('a', id= 'btn_next').get('href')
    except:
        break
    
    if len(df) == 19: 
        #投球数、球種、球速、結果、投手、打者、スコアを取得
        result = df[14].drop('投球数.1', axis=1)
        result['投手'] = df[13].loc[0, '投手']
        result['打者'] = df[13].loc[0, '打者']
        result['スコア'] = '-'.join(map(str, df[1].iloc[:, 1].tolist()))
        
        #投球コースを取得
        Course = soup.find('div', class_= 'bb-allocationChart').find_all('span', class_='bb-icon__ballCircle')
        course = [c['style'] for c in Course]
        result['投球コース'] = course

        #データフレームを結合
        df_result = pd.concat([df_result, result])
    
    #次のページのURLに更新
    url = 'https://baseball.yahoo.co.jp' + next

In [17]:
df_result

,投球数,球種,球速,結果,投手,打者,スコア,投球コース
0,1,ストレート,146km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:11px; left:132px;
1,2,ストレート,146km/h,見逃し,渡邉 勇太朗,辰己 涼介,0-0,top:119px; left:112px;
2,3,スライダー,130km/h,ファウル,渡邉 勇太朗,辰己 涼介,0-0,top:35px; left:72px;
3,4,ストレート,149km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:150px; left:99px;
4,5,カットボール,135km/h,左失,渡邉 勇太朗,辰己 涼介,0-0,top:88px; left:68px;
...,...,...,...,...,...,...,...,...
0,1,チェンジアップ,134km/h,見逃し,ボー・タカハシ,鈴木 大地,4-9,top:136px; left:69px;
1,2,ストレート,-,ファウル,ボー・タカハシ,鈴木 大地,4-9,top:38px; left:94px;
2,3,チェンジアップ,134km/h,ボール [ワンバウンド],ボー・タカハシ,鈴木 大地,4-9,top:165px; left:24px;
3,4,ストレート,150km/h,ボール [すっぽ抜け],ボー・タカハシ,鈴木 大地,4-9,top:0px; left:137px;


In [27]:
#データフレームを用意
df_result = pd.DataFrame()

#試合のIDのようなもの
days = list(range(2021005991,2021005993))
#days = list(range(2021005325,2021005326))
#2021005325が2022年の初めの試合のID(2/26(土)ソフトバンク対オリックス)
#2021006003が2022年7月24日時点で最新の試合(西武対楽天)
#678試合分のデータがある
#days = ['2021005991']
for d in days:
    url = 'https://baseball.yahoo.co.jp/npb/game/' + str(d) + '/score?index=0110100'
    #url = 'https://baseball.yahoo.co.jp/npb/game/2021005991/score?index=0110100'
    
    for i in range(1000):
        #1秒待機
        time.sleep(1)
        #read_htmlでデータを取得
        df = pd.read_html(url, encoding='utf-8')
        #BeautifulSoupでデータを取得
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        #「次へ」ボタンからURLを取得
        try:
            next = soup.find('a', id= 'btn_next').get('href')
        except:
            break

        if len(df) == 19: 
            #投球数、球種、球速、結果、投手、打者、スコアを取得
            result = df[14].drop('投球数.1', axis=1)
            result['投手'] = df[13].loc[0, '投手']
            result['打者'] = df[13].loc[0, '打者']
            result['スコア'] = '-'.join(map(str, df[1].iloc[:, 1].tolist()))

            #投球コースを取得
            Course = soup.find('div', class_= 'bb-allocationChart').find_all('span', class_='bb-icon__ballCircle')
            course = [c['style'] for c in Course]
            result['投球コース'] = course

            #データフレームを結合
            df_result = pd.concat([df_result, result])

        #次のページのURLに更新
        url = 'https://baseball.yahoo.co.jp' + next

In [28]:
df_result

,投球数,球種,球速,結果,投手,打者,スコア,投球コース
0,1,ストレート,146km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:11px; left:132px;
1,2,ストレート,146km/h,見逃し,渡邉 勇太朗,辰己 涼介,0-0,top:119px; left:112px;
2,3,スライダー,130km/h,ファウル,渡邉 勇太朗,辰己 涼介,0-0,top:35px; left:72px;
3,4,ストレート,149km/h,ボール,渡邉 勇太朗,辰己 涼介,0-0,top:150px; left:99px;
4,5,カットボール,135km/h,左失,渡邉 勇太朗,辰己 涼介,0-0,top:88px; left:68px;
...,...,...,...,...,...,...,...,...
4,5,ストレート,152km/h,ボール,モイネロ,安達 了一,7-5,top:21px; left:121px;
5,6,ストレート,152km/h,ファウル,モイネロ,安達 了一,7-5,top:98px; left:88px;
6,7,チェンジアップ,143km/h,四球,モイネロ,安達 了一,7-5,top:168px; left:96px;
0,1,チェンジアップ,137km/h,見逃し,モイネロ,紅林 弘太郎,7-5,top:125px; left:45px;


In [29]:
#df_result.to_csv('game_results_1_2.csv', mode='a', index=False)
df_result.to_csv('game_results_1_2.csv',index=False)